In [119]:
#load and preprocess the data
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


In [20]:
# Replace file paths with the paths to the downloaded files
orders_df = pd.read_csv("../datasets/olist_orders_dataset.csv")
order_items_df = pd.read_csv("../datasets/olist_order_items_dataset.csv")
products_df = pd.read_csv("../datasets/olist_products_dataset.csv")
reviews_df = pd.read_csv("../datasets/olist_order_reviews_dataset.csv")


In [21]:
#checking for missing values 
print(orders_df.isnull().sum())
print(order_items_df.isnull().sum())
print(products_df.isnull().sum())
print(reviews_df.isnull().sum())


order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64
order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64
product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64
review_id                      0
order_id                       0
review_score                   0
review_comment_title       87656
review_comment_message     58247
review_crea

In [22]:
# Impute categorical data with the mode (most frequent category)
products_df['product_category_name'].fillna(products_df['product_category_name'].mode()[0], inplace=True)

# Impute numerical data with the mean
for col in ['product_name_lenght', 'product_description_lenght', 'product_photos_qty']:
    products_df[col].fillna(products_df[col].mean(), inplace=True)

# For very few missing values in product dimensions and weight, use mean
for col in ['product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']:
    products_df[col].fillna(products_df[col].mean(), inplace=True)

# For review comments, replace missing values with 'No Comment'
reviews_df['review_comment_title'].fillna('No Comment', inplace=True)
reviews_df['review_comment_message'].fillna('No Comment', inplace=True)


In [23]:
#after handling the missing values we recheck the datasets to ensure that all missing values have been appropriately addressed
print("Missing values in Product Dataset:")
print(products_df.isnull().sum())

print("\nMissing values in Review Dataset:")
print(reviews_df.isnull().sum())



Missing values in Product Dataset:
product_id                    0
product_category_name         0
product_name_lenght           0
product_description_lenght    0
product_photos_qty            0
product_weight_g              0
product_length_cm             0
product_height_cm             0
product_width_cm              0
dtype: int64

Missing values in Review Dataset:
review_id                  0
order_id                   0
review_score               0
review_comment_title       0
review_comment_message     0
review_creation_date       0
review_answer_timestamp    0
dtype: int64


In [24]:
# Merging the datasets

full_order_df = pd.merge(orders_df, order_items_df, on='order_id', how='left')
full_order_product_df = pd.merge(full_order_df, products_df, on='product_id', how='left')

In [25]:
#creating new features 
#based on the merged data, we want to create new features that can help in our analysis or model building

full_order_product_df['total_value'] = full_order_product_df['price'] + full_order_product_df['freight_value']

# Example: Convert timestamp to datetime and extract useful parts
full_order_product_df['order_purchase_timestamp'] = pd.to_datetime(full_order_product_df['order_purchase_timestamp'])
full_order_product_df['purchase_weekday'] = full_order_product_df['order_purchase_timestamp'].dt.day_name()
full_order_product_df['purchase_hour'] = full_order_product_df['order_purchase_timestamp'].dt.hour


In [26]:
#after we create the new features we do the final check for missing values and data types
# Check for missing values in the new dataframe
print(full_order_product_df.isnull().sum())

# Check data types
print(full_order_product_df.dtypes)

# Save the processed dataframe to a new CSV for easier access in the future
full_order_product_df.to_csv('processed_data.csv', index=False)
    

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 161
order_delivered_carrier_date     1968
order_delivered_customer_date    3229
order_estimated_delivery_date       0
order_item_id                     775
product_id                        775
seller_id                         775
shipping_limit_date               775
price                             775
freight_value                     775
product_category_name             775
product_name_lenght               775
product_description_lenght        775
product_photos_qty                775
product_weight_g                  775
product_length_cm                 775
product_height_cm                 775
product_width_cm                  775
total_value                       775
purchase_weekday                    0
purchase_hour                       0
dtype: int64
order_id                             

In [27]:
# Handling missing values
# Dropping rows where order items are missing
full_order_product_df.dropna(subset=['order_item_id'], inplace=True)

# Filling missing dates with placeholder or imputation
full_order_product_df['order_approved_at'].fillna(method='ffill', inplace=True)  # Example: forward fill

# Converting date columns to datetime
full_order_product_df['order_approved_at'] = pd.to_datetime(full_order_product_df['order_approved_at'])

# Save the processed dataframe
full_order_product_df.to_csv('processed_data.csv', index=False)


#### A simple recommender system based on previously ordered product categories

In [98]:
# Merging datasets to get the user, product, and category information together
merged_df = pd.merge(orders_df, order_items_df, on='order_id')
merged_df = pd.merge(merged_df, products_df, on='product_id')

In [194]:
def recommend_products(customer_id, num_recommendations=5):
    # Find products previously ordered by the customer
    ordered_products = merged_df[merged_df['customer_id'] == customer_id]['product_category_name']
    
    # Recommend other products in the same categories
    recommendations = merged_df[merged_df['product_category_name'].isin(ordered_products)]
    return recommendations['product_id'].unique()[:num_recommendations]

In [195]:
recommend_products(merged_df['customer_id'][0])

array(['87285b34884572647811a353c7ac498a',
       'be021417a6acb56b9b50d3fd2714baa8',
       'a5a0e71a81ae65aa335e71c06261e260',
       '97f1396a5a1f7c07ba51784efdec44b8',
       '978859c6048ded0fa8bf3b9ea8236387'], dtype=object)

#### Recommendations based on Cosine Similarity

In [102]:
# Merge Review Scores
merged_df = pd.merge(full_order_product_df, reviews_df[['order_id', 'review_score']], on='order_id', how='left')

# Create 'Rating' Column
# Handling missing values - fill with average score or a predefined score
default_rating = merged_df['review_score'].mean()
merged_df['rating'] = merged_df['review_score'].fillna(default_rating)

merged_df = pd.merge(full_order_product_df, reviews_df[['order_id', 'review_score']], on='order_id', how='left')

# Step 4: Create the 'Rating' Column
default_rating = merged_df['review_score'].mean()
merged_df['rating'] = merged_df['review_score'].fillna(default_rating)

# Selecting top N products based on the number of orders
top_products = merged_df['product_id'].value_counts().head(2000).index
filtered_df = merged_df[merged_df['product_id'].isin(top_products)]




In [103]:
#  create the pivot table with this filtered dataframe
pivot_table = filtered_df.pivot_table(index='customer_id', columns='product_id', values='rating').fillna(0)
pivot_table_sparse = csr_matrix(pivot_table)

In [93]:
# Compute cosine similarity between items
item_similarity = cosine_similarity(pivot_table_sparse.T, dense_output=False)

# Create a mapping from product IDs to internal indices
unique_product_ids = filtered_df['product_id'].unique()
product_id_mapping = pd.Series(index=unique_product_ids, data=range(len(unique_product_ids)))

In [192]:
def recommend_products_cosine(product_id, top_n=5):
    if product_id not in product_id_mapping.index:
        raise ValueError(f"Product ID '{product_id}' not found in the mapping.")
    
    # Find the internal index of the product
    product_idx = product_id_mapping[product_id_mapping.index == product_id].iloc[0]
    
    print("Product ID:", product_id)
    print("Internal Index:", product_idx)
    
    # Get similarity values
    similarity_values = item_similarity[product_idx].toarray().flatten()
    
    # Get indices of top similar products
    similar_product_indices = similarity_values.argsort()[::-1][1:top_n+1]  # Exclude the product itself
    
    # Convert these indices back to product IDs
    similar_products = product_id_mapping.iloc[similar_product_indices].index
    
    return similar_products

In [193]:
random_product_id = filtered_df['product_id'].sample().iloc[0]
recommendations = recommend_products_cosine(random_product_id)

Product ID: cc5447118c174dcc6456c84ccb29e6f7
Internal Index: 104


#### Recommendations based on Collaborative Filtering

In [ ]:
# Create a sparse matrix for customer-product interactions
customer_ids = pd.factorize(filtered_df['customer_id'])[0]
product_ids = pd.factorize(filtered_df['product_id'])[0]
ratings = filtered_df['rating'].values

pivot_table_sparse = csr_matrix((ratings, (customer_ids, product_ids)), shape=(len(np.unique(customer_ids)), len(np.unique(product_ids))))

# Create a mapping from customer and product IDs to internal indices
customer_id_mapping = pd.Series(index=filtered_df['customer_id'].unique(), data=np.unique(customer_ids))
product_id_mapping = pd.Series(index=filtered_df['product_id'].unique(), data=np.unique(product_ids))

# Compute cosine similarity between items
item_similarity = cosine_similarity(pivot_table_sparse.T, dense_output=False)

In [190]:
def recommend_products_collaborative(customer_id, top_n=5):
    if customer_id not in customer_id_mapping.index:
        raise ValueError(f"Customer ID '{customer_id}' not found in the mapping.")

    # Find the internal index of the customer
    customer_idx = customer_id_mapping[customer_id]

    # Get ratings for the customer
    customer_ratings = pivot_table_sparse[customer_idx].reshape(1, -1)

    print("Dimensions of item_similarity:", item_similarity.shape)
    print("Dimensions of customer_ratings:", customer_ratings.shape)

    # Compute the weighted sum of similarity values
    weighted_sum = item_similarity.dot(customer_ratings.T)

    # Get indices of purchased products
    purchased_product_indices = product_ids[customer_ratings.nonzero()[1]]

    # Set the corresponding values in the weighted_sum to 0
    weighted_sum[purchased_product_indices] = 0
    print(weighted_sum)
    print('--------')
    print(top_n)

    top_n = min(top_n, weighted_sum.shape[0])
    kth = min(top_n, weighted_sum.shape[0]) - 1
    kth = max(0, kth)  # Ensure kth is not negative
    
    if kth < len(weighted_sum.data):
        top_product_indices = np.argpartition(weighted_sum.data, kth)[:kth + 1]
        top_products = product_id_mapping.iloc[weighted_sum.indices[top_product_indices]].index
    else:
        # Handle the case when kth is out of bounds
        top_products = product_id_mapping.index
    
    return top_products[:top_n]

In [191]:
random_customer_id = filtered_df['customer_id'].sample().iloc[0]
recommendations_collaborative = recommend_products_collaborative(random_customer_id)
print("Collaborative Filtering Recommendations:", recommendations_collaborative)

Dimensions of item_similarity: (2000, 2000)
Dimensions of customer_ratings: (1, 2000)
  (287, 0)	0.6114938481177251
  (405, 0)	0.0
  (603, 0)	9.999999999999996
  (1624, 0)	0.016274346579488577
  (1951, 0)	0.11210013037294261
--------
5
Collaborative Filtering Recommendations: Index(['595fac2a385ac33a80bd5114aec74eb8', '595fac2a385ac33a80bd5114aec74eb8',
       '595fac2a385ac33a80bd5114aec74eb8', '595fac2a385ac33a80bd5114aec74eb8',
       '595fac2a385ac33a80bd5114aec74eb8'],
      dtype='object')
